In [30]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.insert(0, os.path.join(os.path.abspath('.'),'..', 'src'))
import tree_utils, ctree

import numpy as np
import scipy as sc
from scipy import stats
import pandas as pd
import pickle

from sklearn.preprocessing import OneHotEncoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data_dir = r'J:\Onderzoek\21-763_rvanes_MiniECG-2-Data\E_ResearchData\2_ResearchData\Parquet'

In [60]:
MIN_MORPHO_PRESENCE = 0.10 # %

Split_col = 'Dataset'
Target_col = 'Diagnosis'
features_to_use = []
ignore_cols = ["Heart Axis Diagnosis"]

In [73]:
DATA = pd.read_parquet(os.path.join(data_dir, 'DATA.parquet'))

morphology_columns = [c for c in DATA.columns if 'morphology' in c]
lead_columns = [c for c in DATA.columns if ('lead' in c) & ('morphology' not in c)]

for c in morphology_columns:
    DATA.loc[:, c] = DATA[c].apply(lambda x: x[0].strip(",").strip(" "))
    DATA.loc[:, c] = DATA[c].apply(lambda x: x if x.strip()!="" else "none")

OneHot = OneHotEncoder(drop=None, sparse_output=False, min_frequency=MIN_MORPHO_PRESENCE)
MorphologyOneHot = pd.DataFrame(data=OneHot.fit_transform(DATA[morphology_columns]), 
                            columns=OneHot.get_feature_names_out(morphology_columns),
                            index=DATA.index)

In [74]:
DATA = DATA.drop(morphology_columns, axis=1)
DATA = pd.concat([DATA, MorphologyOneHot], axis=1)

In [76]:
keep_columns = [c for c in DATA.columns if c not in ignore_cols]
y_train = DATA.loc[DATA[Split_col]=='training', Target_col]
y_test = DATA.loc[DATA[Split_col]=='test', Target_col]
X_train = DATA.loc[DATA[Split_col]=='training', keep_columns].drop([Target_col, Split_col], 
                                                                   axis=1)
X_test = DATA.loc[DATA[Split_col]=='test', keep_columns].drop([Target_col, Split_col],
                                                              axis=1)

# Make tree

In [78]:
rules_path = r'T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\assets\conduction_tree.json'

In [80]:
rules_loader = ctree.LoadRules(rules_path)
processed_rules = rules_loader.get_processed_rules()

In [81]:
# ignore_after and features_to_use_next not yet incorporated.. TODO :)
processed_rules

{'name': 'root-node',
 'feature': 'QRS_duration mean',
 'condition': 'higher_than_or_equal_to',
 'value': 110,
 'children': [{'name': 'root-child-1',
   'feature': 'QRS_duration mean',
   'condition': 'higher_than_or_equal_to',
   'value': 120,
   'pre_condition_value': True},
  {'name': 'root-child-2',
   'feature': [],
   'condition': [],
   'value': [],
   'pre_condition_value': False}]}